In [37]:
import os
import requests
import pandas as pd

toll_traffic_url = 'https://www.mopc.gob.do/media/26614/trafico-de-peaje-rd-vial-2021-2022-noviembre2023.ods'


def load_traffic_url(is_local: bool, folder_name: str = 'data', file_name: str = None) -> str:
    if is_local is False:
        return toll_traffic_url

    if not file_name:
        file_name = toll_traffic_url.split('/')[-1]

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    file_path = f'./{folder_name}/{file_name}'

    if not os.path.isfile(file_path):
        with open(file_path, 'wb') as f:
            f.write(requests.get(toll_traffic_url).content)

    return file_path


df = pd.read_excel(load_traffic_url(False), engine='odf')

df['Estaciones'] = df['Estaciones'].astype('string')
df['Mes '] = df['Mes '].apply(lambda month: month.strip()).astype('string')
stripped_columns = {column: column.strip() for column in df.columns}
df.rename(columns=stripped_columns, inplace=True)

df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

print(df.dtypes)
print(len(df['Estaciones'].unique()))
df.head()

Estaciones          object
Categoria            int64
Tarifa               int64
Total de Trafico     int64
Mes                 object
Año                  int64
dtype: object
15


,Estaciones,Categoria,Tarifa,Total de Trafico,Mes,Año
0,Americas,1,60,520969,Enero,2021
1,Americas,2,120,39235,Enero,2021
2,Americas,3,180,8825,Enero,2021
3,Americas,4,240,17181,Enero,2021
4,Americas,5,300,347,Enero,2021


In [38]:
month_numbers = {month: index+1 for index, month in enumerate(df['Mes'].unique())}
df['Mes'] = df['Mes'].apply(lambda month: month_numbers[month])
print(df['Mes'].unique())


def get_last_day_of_month(year, month):
    return pd.to_datetime(f'{year}-{month}-01') + pd.offsets.MonthEnd(1)


df['Fecha'] = df.apply(lambda row: get_last_day_of_month(row['Año'], row['Mes']), axis=1)
df.drop(columns=['Año', 'Mes'], inplace=True)

df.head()

[ 1  2  3  4  5  6  7  8  9 10 11 12]


,Estaciones,Categoria,Tarifa,Total de Trafico,Fecha
0,Americas,1,60,520969,2021-01-31
1,Americas,2,120,39235,2021-01-31
2,Americas,3,180,8825,2021-01-31
3,Americas,4,240,17181,2021-01-31
4,Americas,5,300,347,2021-01-31


In [39]:
coors_path = './data/locations.csv'
locations_df = pd.read_csv(coors_path)

df = df.merge(locations_df, left_on='Estaciones', right_on='title', how='left')
df.drop(columns=['title'], inplace=True)

df.set_index('Fecha', inplace=True)

df.head()

,Estaciones,Categoria,Tarifa,Total de Trafico,lat,lng
Fecha,,,,,,
2021-01-31,Americas,1,60,520969,18.4531,-69.69142
2021-01-31,Americas,2,120,39235,18.4531,-69.69142
2021-01-31,Americas,3,180,8825,18.4531,-69.69142
2021-01-31,Americas,4,240,17181,18.4531,-69.69142
2021-01-31,Americas,5,300,347,18.4531,-69.69142


In [40]:
import geopandas as gpd

geodf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lng'], df['lat']))
geodf.drop(columns=['lat', 'lng'], inplace=True)
geodf.head()

,Estaciones,Categoria,Tarifa,Total de Trafico,geometry
Fecha,,,,,
2021-01-31,Americas,1,60,520969,POINT (-69.69142 18.45310)
2021-01-31,Americas,2,120,39235,POINT (-69.69142 18.45310)
2021-01-31,Americas,3,180,8825,POINT (-69.69142 18.45310)
2021-01-31,Americas,4,240,17181,POINT (-69.69142 18.45310)
2021-01-31,Americas,5,300,347,POINT (-69.69142 18.45310)
